<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/lesson9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 9. Языковое моделирование

Задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
path_to_file = '/content/evgenyi_onegin.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [5]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text[:100], len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         ',
 286984,
 286984)

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [13]:
# Batch size
BATCH_SIZE = 50

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(50, 100), dtype=tf.int64, name=None), TensorSpec(shape=(50, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (50, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (50, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (50, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (50, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (50, None, 1024)          8392704   
                                                                 
 dense (Dense)               (50, None, 131)           134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899
No

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
!rm -rf ./training_checkpoints

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

Начнем с 50 эпох

In [22]:
EPOCHS = 50

In [23]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
56/56 [==============================] - 27s 310ms/step - loss: 2.4408
Epoch 2/50
56/56 [==============================] - 18s 322ms/step - loss: 2.2410
Epoch 3/50
56/56 [==============================] - 20s 339ms/step - loss: 2.2396
Epoch 4/50
56/56 [==============================] - 19s 337ms/step - loss: 2.2061
Epoch 5/50
56/56 [==============================] - 19s 325ms/step - loss: 2.0377
Epoch 6/50
56/56 [==============================] - 19s 325ms/step - loss: 1.7392
Epoch 7/50
56/56 [==============================] - 19s 332ms/step - loss: 1.6495
Epoch 8/50
56/56 [==============================] - 20s 356ms/step - loss: 1.5564
Epoch 9/50
56/56 [==============================] - 19s 331ms/step - loss: 1.4960
Epoch 10/50
56/56 [==============================] - 19s 328ms/step - loss: 1.4428
Epoch 11/50
56/56 [==============================] - 19s 331ms/step - loss: 1.4157
Epoch 12/50
56/56 [==============================] - 19s 335ms/step - loss: 1.3959
Epoch 13/50
5

In [24]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_48'

In [25]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Температура 1

In [39]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.0001

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Температура 1

In [34]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже ,  ее,м н у
в  н  . .я  ..   л н ,ен
..ш , ,

н  в ую, д )!  е  .в о 
п н : л 
н  н  .
 л  у.  ейл с н      я
    ж 
. ясг. е. р ,  л ;;
к  ян  ю  йо. ) н . ер .ян .  м ин у .ч 
; я щ  , . и    е л 
е и.м ео   о, л  м сеьед   л .л н 

,г   н  у н н   е н           .ш , ;.иам   
  н  е  юд яо .
  в н 

  м н р .я;л 
.м 
. л
р     н.  .:ее г
и.р н   
  к 
л. л н  в 
м 
н   ин
 яр ,л л   я .я 
яе  ;  
 ?яе,и яя,н н.  к     к и.е  е      н
 еяд, 
   н  и. ив  ек    . а!     . !   . :н   р яя н н ,
!


Температура 0.1

In [38]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже                 н        н   н      н                  н        н н        н         м          н       н            ен  е       н           н         н        н       н          н н                н      н            н            р н            н н         н           н н          н        н          н            н н        н          н             н н                    н     н        м           н        н      н          н           н  м             н н           н         н        н        


Температура 0.0001

In [40]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже                     н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н             н   


Результат не впечатляет, попробуем обучить на 150 эпохах

In [41]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [44]:
EPOCHS=150

In [45]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/150
56/56 [==============================] - 24s 350ms/step - loss: 2.4987
Epoch 2/150
56/56 [==============================] - 19s 337ms/step - loss: 2.2296
Epoch 3/150
56/56 [==============================] - 18s 323ms/step - loss: 1.9056
Epoch 4/150
56/56 [==============================] - 19s 326ms/step - loss: 1.7205
Epoch 5/150
56/56 [==============================] - 19s 336ms/step - loss: 1.5822
Epoch 6/150
56/56 [==============================] - 19s 338ms/step - loss: 1.4991
Epoch 7/150
56/56 [==============================] - 19s 333ms/step - loss: 1.4430
Epoch 8/150
56/56 [==============================] - 20s 348ms/step - loss: 1.4051
Epoch 9/150
56/56 [==============================] - 19s 332ms/step - loss: 1.3948
Epoch 10/150
56/56 [==============================] - 19s 334ms/step - loss: 1.3718
Epoch 11/150
56/56 [==============================] - 19s 335ms/step - loss: 1.3528
Epoch 12/150
56/56 [==============================] - 19s 334ms/step - loss: 1.3573
E

In [46]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_150'

In [47]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [52]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.0001

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Температура 1

In [49]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже для света,
                        Забвенье жизни в бурях света,
                        Беседу слышит за собой,
                      И она в сенях;
                        За ней ленивей волочусь.
                                    Все думать, думать об одном
                        И день и ночь проводит
                                 Он занят Ольгою своей.
                        Летучие листки альбома
                       За ним сначала и не мог
                        Кто прежней Тани


Температура 0.1

In [51]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже для света,
                        Гут
                        И для Татьяны наконец
                        Его с разрозненной "Мальвиной вешно развалась,
                                                                                                                                                                                                                                                                                                                                                        


Температура 0.0001

In [53]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже для света,
                        Хоть иногда и самом
                        И в самом ужасе она:
                        Так нас природа сотворила,
                                                                                                                                                                                                                                                                                                                                                             


Увеличение количества батчей существенно повлияло на качество модели